In [1]:
import os
import pandas as pd
import spacy
import csv
from spacy.util import minibatch, compounding
from sklearn.model_selection import train_test_split
import random
from spacy.util import minibatch, compounding
from spacy.training.example import Example


2023-06-12 15:05:52.354961: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-12 15:05:52.392075: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-12 15:05:52.970257: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

You should consider upgrading via the '/opt/conda/bin/python3.10 -m pip install --upgrade pip' command.
2023-06-10 22:05:01.287433: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 18.7 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
# data
from subprocess import check_output
print(check_output(["ls", "/home/jovyan/work/data/label_data"]).decode("utf8"))

extra
ner.csv
nerReformat.csv
ner_test_data.csv
ner_train_data.csv



In [11]:
def reformat_data_spacy_format(data):
    grouped = data.groupby("sentence_idx").apply(
        lambda s: [(w, t) for w, t in zip(s["word"].values.tolist(), s["tag"].values.tolist())])
    sentences = [s for s in grouped]
    data_encoded = []
    for s in sentences:
        full_s = ""
        start_index = 0
        last_index = 0
        index_counter = 0
        entities = []
        for w in s:
            full_s += str(w[0]) + " "
            entities.append((index_counter, index_counter + len(str(w[0])), w[1]))
            index_counter += len(str(w[0])) + 1
        data_encoded.append((full_s, {"entities": entities}))

    return data_encoded


In [12]:
train_df = pd.read_csv("/home/jovyan/work/data/label_data/ner_train_data.csv")
test_df = pd.read_csv("/home/jovyan/work/data/label_data/ner_test_data.csv")

train_data = reformat_data_spacy_format(train_df)
test_data = reformat_data_spacy_format(test_df)


In [5]:
# grouped = dframe.groupby("sentence_idx").apply(lambda s: [(w, t) for w, t in zip(s["word"].values.tolist(),s["tag"].values.tolist())])
# sentences = [s for s in grouped]

In [6]:
# test = [[('TokyoiTower', 'O'),
#   ('of', 'O'),
#   ('demonstrators', 'O'),
#   ('have', 'O'),
#   ('marched', 'O'),
#   ('through', 'O'),
#   ('London', 'B-geo'),
#   ('to', 'O'),
#   ('protest', 'O')],[
#   ('the', 'O'),
#   ('war', 'O'),
#   ('in', 'O'),
#   ('Iraq', 'B-geo')]]
# data = []
# for s in sentences:
#     full_s = ""
#     start_index = 0
#     last_index = 0
#     index_counter = 0
#     entities = []
#     for w in s:
#         full_s += str(w[0]) + " "
#         entities.append((index_counter, index_counter+len(str(w[0])),w[1]))
#         index_counter += len(str(w[0])) + 1
#
#
#
#     data.append((full_s,{"entities":entities}))
#
# train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


In [7]:
# Initialize a new blank spaCy model
nlp = spacy.load("en_core_web_sm")

# Create the NER component
ner = nlp.get_pipe("ner")

In [13]:
# Add custom tags to the NER pipeline
for _, annotations in train_data:
    for entity in annotations.get("entities"):
        ner.add_label(entity[2])

# Disable unnecessary pipeline components
pipe_exceptions = ["ner"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]


In [14]:
# TRAINING THE MODEL
with nlp.disable_pipes():
    # Training iterations...
    for iteration in range(30):
        # Shuffle training examples before every iteration
        random.shuffle(train_data)
        losses = {}
        examples = []

        # Create Example objects from training data
        for text, annotation in train_data:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotation)
            examples.append(example)

        # Batch up the examples using spaCy's minibatch
        batches = minibatch(examples, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            nlp.update(batch, drop=0.5, losses=losses)

        print("Losses:", losses)


/opt/conda/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "He says they and about 300 party supporters are tr..." with entities "[(0, 2, 'O'), (3, 7, 'O'), (8, 12, 'O'), (13, 16, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "" And I think they 'll want a one-stop shop in ter..." with entities "[(0, 1, 'O'), (2, 5, 'O'), (6, 7, 'O'), (8, 13, 'O...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 41501.814158223424}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 40232.1288467212}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 39687.296533950335}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 38902.52636717124}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 38247.48940987092}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 37757.3574880842}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 37211.016008727325}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 36886.55171803751}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 36303.07585804052}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 36291.707210676745}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 35863.30254350039}
Losses: {'tok2vec': 0.0, 'tagger': 0.0, 'parser': 0.0, 'ner': 35522.94983535848}
Losses: {'tok2vec': 0.0, '

In [16]:
# Save the trained model
output_dir = "/home/jovyan/work/data/output/spacyModel"
nlp.to_disk(output_dir)

In [18]:
from sklearn.metrics import precision_score, recall_score, f1_score

test_examples = []
for text, annotation in test_data:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotation)
    test_examples.append(example)

# Evaluate the model on the test set
scores = nlp.evaluate(test_examples)
print("Test Scores:", scores)

# Calculate accuracy, precision, recall, and F1 score for each tag
tag_metrics = {}
for tag, values in scores['ents_per_type'].items():
    precision = round(values['p'] * 100, 2)  # Convert precision to percentage and round to 2 decimal places
    recall = round(values['r'] * 100, 2)  # Convert recall to percentage and round to 2 decimal places
    f1 = round(values['f'] * 100, 2)  # Convert F1 score to percentage and round to 2 decimal places

    tag_metrics[tag] = {'accuracy': precision, 'precision': precision, 'recall': recall, 'f1': f1}

# Print metrics for each tag
print("Metrics per Tag:")
for tag, metrics in tag_metrics.items():
    print("- Tag:", tag)
    print("  Accuracy:", metrics['accuracy'], "%")
    print("  Precision:", metrics['precision'], "%")
    print("  Recall:", metrics['recall'], "%")
    print("  F1 Score:", metrics['f1'], "%")
    print()


Test Scores: {'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'tag_acc': None, 'sents_p': 1.0, 'sents_r': 1.0, 'sents_f': 1.0, 'dep_uas': None, 'dep_las': None, 'dep_las_per_type': None, 'pos_acc': None, 'morph_acc': None, 'morph_micro_p': None, 'morph_micro_r': None, 'morph_micro_f': None, 'morph_per_feat': None, 'lemma_acc': None, 'ents_p': 0.98170082401679, 'ents_r': 0.981595596186919, 'ents_f': 0.981648207281879, 'ents_per_type': {'O': {'p': 0.995110065724318, 'r': 0.9948595482309391, 'f': 0.9949847912087911}, 'B-geo': {'p': 0.897754661930737, 'r': 0.9442294863242161, 'f': 0.9204057744830277}, 'B-gpe': {'p': 0.9754989453188382, 'r': 0.9471445450964947, 'f': 0.9611126653610967}, 'B-org': {'p': 0.8972598035849186, 'r': 0.8049907578558225, 'f': 0.8486245980968464}, 'I-geo': {'p': 0.8841673502871206, 'r': 0.903588195841717, 'f': 0.8937722862592254}, 'B-tim': {'p': 0.9228896597894224, 'r': 0.9312208302628346, 'f': 0.9270365275867339}, 'I-org': {'p': 0.936415260337519, 

In [19]:
entity_metrics = scores['ents_per_type']

precision_sum = 0.0
recall_sum = 0.0
f_score_sum = 0.0
num_entity_types = len(entity_metrics)

for entity_type, metrics in entity_metrics.items():
    precision_sum += metrics['p']
    recall_sum += metrics['r']
    f_score_sum += metrics['f']

general_precision = precision_sum / num_entity_types
general_recall = recall_sum / num_entity_types
general_f_score = f_score_sum / num_entity_types

print("General Precision:", general_precision)
print("General Recall:", general_recall)
print("General F-Score:", general_f_score)

General Precision: 0.8608406012098615
General Recall: 0.7423703228858265
General F-Score: 0.784462621930496
